In [2]:
#import dependencies 
import pandas as pd 
import requests 
import json 
import snowflake.connector
import config 
import os 
import time 
import numpy as np
import functions 

In [6]:
#cereate connection to specigic database/schema 
con = snowflake.connector.connect(
    user=config.keys['user'],
    password=config.keys['password'],
    account=config.keys['account'],
    warehouse = 'WAREHOUSE_BLUE',
    database  = 'DATABASE_BLUE',
    schema = 'SCHEMA_BLUE'
)

In [1]:
#page through 
#get dataset size then get how many pages 
#keep track of pages have acount to reference 

https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2023/869v-vr48

In [9]:
samp1 = requests.get(f'https://data.cityofnewyork.us/resource/869v-vr48.json?$limit=10')

In [5]:
request_limit = 10
request_offset = 0
samp = requests.get(f'https://data.cityofnewyork.us/resource/869v-vr48.json?$limit={request_limit}&$offset={request_offset}&$order=summons_number')

In [13]:
request_limit = 10
request_offset = 5
samp2 = requests.get(f'https://data.cityofnewyork.us/resource/869v-vr48.json?$limit={request_limit}&$offset={request_offset}&$order=summons_number')

- the offset needs to increase by whatever the limit is 
- need to get dataset size 

In [14]:
df_samp2 = pd.DataFrame(samp2.json())

In [11]:
df_samp1 = pd.DataFrame(samp.json())

In [23]:
#testing offset 
print(df_samp2.iloc[0].summons_number == df_samp1.iloc[5].summons_number)
print(df_samp2.iloc[1].summons_number == df_samp1.iloc[6].summons_number)
print(df_samp2.iloc[2].summons_number == df_samp1.iloc[7].summons_number)
print(df_samp2.iloc[3].summons_number == df_samp1.iloc[8].summons_number)
print(df_samp2.iloc[4].summons_number == df_samp1.iloc[9].summons_number)

True
True
True
True
True


In [7]:
#total number of pagees for dataset
pages = 

#current / latest dataset page number to be requested / retrieved 
page_num = 0